<div class = "row">
    <div class = "colums">
        <img src="..\trecslogo.png" align="left" alt="Drawing" width ="60"/>    
    </div>
    <div class = "colums">
        <img src="..\asrlogo.png" align="right" alt="Drawing" width ="175"/>
    </div>    
</div>

# SWAPTION

## Algemeen

### Import en instellingen

In [1]:
import pandas as pd
import numpy as np
import datetime
import math
from scipy.optimize import fsolve
from scipy.stats import norm
from bokeh.plotting import figure, output_file, ColumnDataSource, output_notebook
from bokeh.models import HoverTool, NumeralTickFormatter, FactorRange
from bokeh.io import show
output_notebook(hide_banner=True)

Links uitlijnen tabellen

In [2]:
%%html
<style>
    table {
        display: inline-block
    }
</style>

Bij printen van een dataframe wordt slechts een beperkt aantal rijen getoond.

In [3]:
pd.set_option('display.max_rows', 10)

## Initialisatie parameters

In [4]:
# data
datePricingPrimo = '20220331' # in dit notebook veronderstel ik dat de actuele datum dezelfde is als de kwartaal datum
datePricingUltimo = '20220430'
datePricingPrimo = datetime.datetime.strptime(datePricingPrimo, '%Y%m%d')
datePricingUltimo = datetime.datetime.strptime(datePricingUltimo, '%Y%m%d')

# naam van de basis curve
curveNameBasis = 'FairValue'

# Bachelier model (instead of H&W)
Bachelier = True

# Haug formula for cash sattled swaptions assumes that 6 month compounded swap rate used as the discounting rate
m = 2

## Importeer en bewerk data

### Rentecurves per kwartaal en actueel
In de TRT is een begin gemaakt om te rekenen met de dual curve methodiek, dus op basis van de zero swap curve en de zero EONIA curve. Dit moet in TRT nog verder worden uitgewerkt en is nog niet operationeel. Dit notebook is daarom vooralsnog obv single curve methodiek (net zoals TRT).

#### Rentecurves primo

In [5]:
df_curvesPrimo = pd.read_excel(r"curvebestanden/curvesAssetsM03.xlsx", decimal = '.')

pd.options.display.float_format = '{:,.8f}'.format
df_curvesPrimo

,Jaar,Currency,FairValue,swap.cra.zero.va.down,swap.cra.zero.va.up,SII_basis,SII_Yield_Curve_down,SII_Yield_Curve_up,SII_basis.EQUITY_TYPE_1,SII_basis.EQUITY_TYPE_2,...,swap.cra.eur-stylized-1.zero.va.sw345.down345,swap.cra.eur-stylized-1.zero.va.sw345.up345,swap.cra.dnb,swap.cra.dnb.up,swap.cra.dnb.down,swap.cra.zero.va-ratio50.sw270,swap.cra.zero.va-ratio50.sw270.down270,swap.cra.zero.va-ratio50.sw270.up270,swap.cra.zero.va-ratio50.sw270_Currency_Up,swap.cra.zero.va-ratio50.sw270_Currency_Down
0,1,EUR,-0.00080869,-0.00080869,0.00919131,-0.00080869,-0.00080869,0.00919131,-0.00080869,-0.00080869,...,-0.00080776,0.00919224,-0.00080869,0.00919131,-0.00080869,-0.00080869,-0.00080869,0.00919131,-0.00080869,-0.00080869
1,2,EUR,0.00537754,0.00253036,0.01537754,0.00537754,0.00254137,0.01537754,0.00537754,0.00537754,...,0.00254794,0.01538412,0.00537754,0.01537754,0.00253036,0.00537754,0.00254137,0.01537754,0.00537754,0.00537754
2,3,EUR,0.00806788,0.00411486,0.01806788,0.00806788,0.00412935,0.01806788,0.00806788,0.00806788,...,0.00413130,0.01806983,0.00806788,0.01806788,0.00411486,0.00806788,0.00412935,0.01806788,0.00806788,0.00806788
3,4,EUR,0.00926595,0.00510382,0.01926595,0.00926595,0.00511846,0.01926595,0.00926595,0.00926595,...,0.00511910,0.01926659,0.00926595,0.01926595,0.00510382,0.00926595,0.00511846,0.01926595,0.00926595,0.00926595
4,5,EUR,0.00990248,0.00577112,0.01990248,0.00990248,0.00578518,0.01990248,0.00990248,0.00990248,...,0.00578534,0.01990264,0.00990248,0.01990248,0.00577112,0.00990248,0.00578518,0.01990248,0.00990248,0.00990248
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,EUR,0.00723252,0.00541209,0.01723252,0.00723252,0.00162101,0.01723252,0.00723252,0.00723252,...,0.00160880,0.01723288,0.00723252,0.01723252,0.00541209,0.00723252,0.00273378,0.01723252,0.00723252,0.00723252
96,97,EUR,0.00723252,0.00541209,0.01723252,0.00723252,0.00160777,0.01723252,0.00723252,0.00723252,...,0.00159568,0.01723288,0.00723252,0.01723252,0.00541209,0.00723252,0.00272451,0.01723252,0.00723252,0.00723252
97,98,EUR,0.00723252,0.00541209,0.01723252,0.00723252,0.00159479,0.01723252,0.00723252,0.00723252,...,0.00158284,0.01723288,0.00723252,0.01723252,0.00541209,0.00723252,0.00271543,0.01723252,0.00723252,0.00723252
98,99,EUR,0.00723252,0.00541209,0.01723252,0.00723252,0.00158208,0.01723252,0.00723252,0.00723252,...,0.00157025,0.01723288,0.00723252,0.01723252,0.00541209,0.00723252,0.00270653,0.01723252,0.00723252,0.00723252


In [6]:
# curveNamesPrimo = list(df_curves.columns)[2:]
curveNamesPrimo = ['FairValue', 'swap.cra.zero.va.down']
curveNamesPrimo

['FairValue', 'swap.cra.zero.va.down']

#### Rentecurves ultimo

In [7]:
df_curvesUltimo = pd.read_excel(r"curvebestanden/curvesAssetsM04.xlsx", decimal = '.')
df_curvesUltimo

,Jaar,Currency,FairValue,swap.cra.zero.va.down,swap.cra.zero.va.up,SII_basis,SII_Yield_Curve_down,SII_Yield_Curve_up,SII_basis.EQUITY_TYPE_1,SII_basis.EQUITY_TYPE_2,...,swap.cra.eur-pc-1.zero.va.sw345.up345,swap.cra.eur-pc-2.zero.va.sw345.down345,swap.cra.eur-pc-2.zero.va.sw345.up345,swap.cra.eur-pc-3.zero.va.sw345.down345,swap.cra.eur-pc-3.zero.va.sw345.up345,swap.cra.eur-stylized-1.zero.va.sw345.down345,swap.cra.eur-stylized-1.zero.va.sw345.up345,swap.cra.dnb,swap.cra.dnb.up,swap.cra.dnb.down
0,1,EUR,0.00259943,0.00139120,0.01259943,0.00259943,0.00139193,0.01259943,0.00259943,0.00259943,...,0.01725770,-0.00356682,0.00643318,0.00102778,0.01115760,0.00138902,0.01259652,0.00259943,0.01259943,0.00139120
1,2,EUR,0.00969469,0.00402633,0.01969469,0.00969469,0.00405226,0.01969469,0.00969469,0.00969469,...,0.02438614,0.00227814,0.01462605,0.00375812,0.01884818,0.00405780,0.01970023,0.00969469,0.01969469,0.00402633
2,3,EUR,0.01243270,0.00606463,0.02243270,0.01243270,0.00609550,0.02243270,0.01243270,0.01243270,...,0.02735691,0.00427634,0.01830597,0.00597806,0.02216739,0.00609468,0.02243187,0.01243270,0.02243270,0.00606463
3,4,EUR,0.01369587,0.00734868,0.02369587,0.01369587,0.00737977,0.02369587,0.01369587,0.01369587,...,0.02860229,0.00572066,0.02038351,0.00744146,0.02382117,0.00737756,0.02369367,0.01369587,0.02369587,0.00734868
4,5,EUR,0.01448096,0.00827133,0.02448096,0.01448096,0.00830175,0.02448096,0.01448096,0.01448096,...,0.02918091,0.00687420,0.02184220,0.00851795,0.02488322,0.00829919,0.02447839,0.01448096,0.02448096,0.00827133
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,EUR,0.01085452,0.00815297,0.02085452,0.01085452,0.00497472,0.02085452,0.01085452,0.01085452,...,0.02554476,0.00616606,0.02209709,0.00409863,0.01999342,0.00496161,0.02085513,0.01085452,0.02085452,0.00815297
96,97,EUR,0.01085452,0.00815297,0.02085452,0.01085452,0.00496423,0.02085452,0.01085452,0.01085452,...,0.02554476,0.00615610,0.02209709,0.00408829,0.01999342,0.00495127,0.02085513,0.01085452,0.02085452,0.00815297
97,98,EUR,0.01085452,0.00815297,0.02085452,0.01085452,0.00495395,0.02085452,0.01085452,0.01085452,...,0.02554476,0.00614634,0.02209709,0.00407816,0.01999342,0.00494113,0.02085513,0.01085452,0.02085452,0.00815297
98,99,EUR,0.01085452,0.00815297,0.02085452,0.01085452,0.00494388,0.02085452,0.01085452,0.01085452,...,0.02554476,0.00613677,0.02209709,0.00406824,0.01999342,0.00493119,0.02085513,0.01085452,0.02085452,0.00815297


In [8]:
# curveNamesUltimo = list(df_curves.columns)[2:]
curveNamesUltimo = ['FairValue', 'swap.cra.zero.va.down']
curveNamesUltimo

['FairValue', 'swap.cra.zero.va.down']

### IMW bestand

CIC codes swaptions:

- B6 = Call options granting its owner the right but not the obligation to enter into a long position in an underlying swap, i.e., enter into a swap where the owner pays the fixed leg and receive the floating leg
- C6 = Put options granting its owner the right but not the obligation to enter into a short position in an underlying swap, i.e., enter into a swap in which the owner will receive the fixed leg, and pay the floating leg

#### IMW bestand primo

In [9]:
col_list = ['Reporting Date', 'Ecs Cons Ecap asr', 'Portfolio Id', 'Cic Id Ll', 'Market Value EUR LL', 'BalNomVal LT', 'Strike Price Laagste Level', 'Maturity Call LL', 'Maturity LL', 'Security Id Ll', 'Security Type Ll', 'Volatility dim']
df_swaptionsIMWPrimo = pd.read_csv(r"imwbestand/2022M03 Adjusted IMW Weekupdate 202205131221.csv", usecols = col_list, sep = ";", decimal = '.', encoding= 'unicode_escape', low_memory=False)

In [10]:
df_swaptionsIMWPrimo = df_swaptionsIMWPrimo.loc[df_swaptionsIMWPrimo['Security Type Ll'] == 'SWAPTION']
df_swaptionsIMWPrimo = df_swaptionsIMWPrimo.reset_index(drop=True)

In [11]:
df_swaptionsIMWPrimo['dateIMW'] = pd.to_datetime(df_swaptionsIMWPrimo['Reporting Date'], format='%Y%m%d')
df_swaptionsIMWPrimo['dateSwapExpiry'] = pd.to_datetime(df_swaptionsIMWPrimo['Maturity LL'], format='%Y%m%d')
df_swaptionsIMWPrimo['dateSwaptionExpiry'] = pd.to_datetime(df_swaptionsIMWPrimo['Maturity Call LL'], format='%Y%m%d')
df_swaptionsIMWPrimo = df_swaptionsIMWPrimo.drop(['Reporting Date', 'Security Type Ll', 'Maturity LL', 'Maturity Call LL'], axis = 1)

In [12]:
df_swaptionsIMWPrimo.rename(columns = { \
                              'Cic Id Ll':'cicCode', \
                              'Ecs Cons Ecap asr': 'company', \
                              'Security Id Ll':'securityId', \
                              'Portfolio Id':'portfolio', \
                              'Market Value EUR LL':'marketValueIMW', \
                              'BalNomVal LT':'nominalValue', \
                              'Volatility dim':'volatilityPercentageIMW', \
                              'Strike Price Laagste Level':'strikePercentage' \
                             }, inplace = True)
pd.options.display.float_format = '{:,.2f}'.format
df_swaptionsIMWPrimo

,company,portfolio,cicCode,securityId,marketValueIMW,nominalValue,volatilityPercentageIMW,strikePercentage,dateIMW,dateSwapExpiry,dateSwaptionExpiry
0,2523_ECAP,BELAMP,XLB6,3350972,"2,332,319.63","50,000,000.00",0.58,2.75,2022-03-31,2068-06-03,2038-06-01
1,2523_ECAP,BELAMP,XLC6,3350574,"965,615.72","2,000,000.00",1.43,4.00,2022-03-31,2042-09-21,2022-09-19
2,2523_ECAP,BEL177,XLC6,3350415,"1,454,048.72","3,000,000.00",1.54,4.00,2022-03-31,2042-07-28,2022-07-26
3,2523_ECAP,BELAMP,XLB6,3350971,"3,485,393.65","50,000,000.00",0.56,2.10,2022-03-31,2068-06-03,2038-06-01
4,2523_ECAP,BELAMP,XLB6,3350980,"1,790,950.35","35,000,000.00",0.57,2.60,2022-03-31,2068-07-29,2038-07-27
...,...,...,...,...,...,...,...,...,...,...,...
48,2523_ECAP,BELAMP,XLB6,3350756,"86,639.40","38,000,000.00",1.26,4.00,2022-03-31,2038-07-07,2023-07-05
49,2523_ECAP,BELAMP,XLB6,3350982,"2,406,877.36","50,000,000.00",0.58,2.70,2022-03-31,2068-08-04,2038-08-02
50,2523_ECAP,BELAMP,XLB6,3350983,"2,719,139.91","50,000,000.00",0.57,2.50,2022-03-31,2068-08-05,2038-08-03
51,2523_ECAP,BELAMP,XLB6,3350967,"5,318,994.74","125,000,000.00",0.59,2.90,2022-03-31,2068-05-26,2038-05-24


#### IMW bestand ultimo

In [13]:
df_swaptionsIMWUltimo = pd.read_csv(r"imwbestand/2022M04 Adjusted IMW Weekupdate 202205301236.csv", usecols = col_list, sep = ";", decimal = '.', encoding= 'unicode_escape', low_memory=False)

In [14]:
df_swaptionsIMWUltimo = df_swaptionsIMWUltimo.loc[df_swaptionsIMWUltimo['Security Type Ll'] == 'SWAPTION']
df_swaptionsIMWUltimo = df_swaptionsIMWUltimo.reset_index(drop=True)

In [15]:
df_swaptionsIMWUltimo['dateIMW'] = pd.to_datetime(df_swaptionsIMWUltimo['Reporting Date'], format='%Y%m%d')
df_swaptionsIMWUltimo['dateSwapExpiry'] = pd.to_datetime(df_swaptionsIMWUltimo['Maturity LL'], format='%Y%m%d')
df_swaptionsIMWUltimo['dateSwaptionExpiry'] = pd.to_datetime(df_swaptionsIMWUltimo['Maturity Call LL'], format='%Y%m%d')
df_swaptionsIMWUltimo = df_swaptionsIMWUltimo.drop(['Reporting Date', 'Security Type Ll', 'Maturity LL', 'Maturity Call LL'], axis = 1)

In [16]:
df_swaptionsIMWUltimo.rename(columns = { \
                              'Cic Id Ll':'cicCode', \
                              'Ecs Cons Ecap asr': 'company', \
                              'Security Id Ll':'securityId', \
                              'Portfolio Id':'portfolio', \
                              'Market Value EUR LL':'marketValueIMW', \
                              'BalNomVal LT':'nominalValue', \
                              'Volatility dim':'volatilityPercentageIMW', \
                              'Strike Price Laagste Level':'strikePercentage' \
                             }, inplace = True)
pd.options.display.float_format = '{:,.2f}'.format
df_swaptionsIMWUltimo

,company,portfolio,cicCode,securityId,marketValueIMW,nominalValue,volatilityPercentageIMW,strikePercentage,dateIMW,dateSwapExpiry,dateSwaptionExpiry
0,2523_ECAP,BELAMP,XLB6,SWOP3350992,"4,691,587.59","25,000,000.00",0.51,0.50,2022-04-30,2070-03-21,2040-03-19
1,2523_ECAP,BELAMP,XLB6,3350967,"5,962,038.17","125,000,000.00",0.60,2.90,2022-04-30,2068-05-26,2038-05-24
2,2523_ECAP,BELAMP,XLB6,3350952,"7,356,626.75","100,000,000.00",0.56,2.10,2022-04-30,2067-07-21,2037-07-17
3,2523_ECAP,BELAMP,XLB6,3350974,"2,300,855.42","41,400,000.00",0.59,2.50,2022-04-30,2063-06-16,2038-06-14
4,2523_ECAP,BELAMP,XLB6,3350955,"6,473,481.65","100,000,000.00",0.59,2.40,2022-04-30,2067-08-05,2037-08-03
...,...,...,...,...,...,...,...,...,...,...,...
48,2523_ECAP,BELAMP,XLB6,SWOP3351024,"6,965,135.77","50,000,000.00",0.51,1.00,2022-04-30,2071-02-13,2041-02-11
49,2523_ECAP,BELAMP,XLB6,3350962,"10,223,462.83","140,000,000.00",0.58,2.20,2022-04-30,2067-09-07,2037-09-03
50,2523_ECAP,BELAMP,XLB6,3350970,"2,851,131.09","50,000,000.00",0.58,2.60,2022-04-30,2068-06-01,2038-05-28
51,2523_ECAP,BELAMP,XLB6,SWOP3351010,"9,370,676.53","50,000,000.00",0.51,0.50,2022-04-30,2070-04-11,2040-04-09


## Documentatie
In dit document staat de methode voor het berekenen van de swaptions beschreven. 

## Functie voor pricing swaptions

In [17]:
def getPriceModel(curve, swaptionType, cashSettled, tenor, expiry, volatility, strike):

#     print('swaptionType', swaptionType)
#     print('cashSettled', cashSettled)
#     print('tenor', round(tenor))
#     print('expiry', expiry)
#     print('volatility', volatility)
#     print('strike', strike)
     
    # make discounts from interest rate curve
    discounts = [1]
    for t in range(1, 100):
        discounts.append(1*(1+curve[t-1])**(-t))
    
    # make maturities for swap fixed cashflows (assumption is 6 months payment)
    maturitiesSwap = [] 
    for t in range(0, round(tenor)*2+1):
        maturitiesSwap.append(expiry+t*0.5)
    
    # make discounts related to swap fixed cashflows via linear interpolation
    maturities = range(0, 100)
    discountsSwap = np.interp(maturitiesSwap, maturities, discounts)
#     print('discountsSwap', discountsSwap[0])
    
    # calculate the forward rate with maturity equal to tenor and starting at expiry date
    forwardRate = (discountsSwap[0]/discountsSwap[-1])**(1/round(tenor))-1
#     print('forwardRate', forwardRate)
    
    if Bachelier:
        d = (forwardRate-strike)/(volatility*math.sqrt(expiry))
        z = 1
        if swaptionType == 'receiver':
            z = -1
        price = volatility*math.sqrt(expiry)*norm.pdf(d)+z*(forwardRate-strike)*norm.cdf(z*d)
        if cashSettled:
            factor = discountsSwap[0]*((1-1/(1+forwardRate/m)**(m*round(tenor)))/forwardRate)
            priceModel = price*factor
        else:
            sumDiscount = 0
            for t in range(1, len(discountsSwap)):
                sumDiscount += discountsSwap[t]/m
            priceModel = price*sumDiscount
    else: # Hull & White
        raise ValueError('Hull & White model is not yet implemented!')

    return priceModel

In [18]:
def calibrateImpliedVolatilityModel(impliedVolatilityModel, *data):
    priceIMW = data[0]
    priceModel = getPriceModel(data[6], data[1], data[2], data[3], data[4], impliedVolatilityModel, data[5])
#     print(priceModel, priceIMW, impliedVolatilityModel)
    return priceModel-priceIMW

## Classes voor swaptions

### Class voor waardering swaption

In [19]:
class Swaption:

    # Initialiseren van class argumenten obv meegegeven data
    # Bij het aanmaken van een nieuw object worden deze meteen geïnitialiseerd

    def __init__(self, dateIMW, company, portfolio, cicCode, securityId, marketValueIMW, nominalValue, volatilityPercentageIMW, strikePercentage, dateSwapExpiry, dateSwaptionExpiry):
        self.dateIMW = dateIMW
        self.company = company
        self.portfolio = portfolio
        self.cicCode = cicCode
        self.securityId = securityId
        self.marketValueIMW = marketValueIMW
        self.nominalValue = nominalValue
        self.volatilityIMW = volatilityPercentageIMW/100
        self.strike = strikePercentage/100
        self.dateSwapExpiry = dateSwapExpiry
        self.dateSwaptionExpiry = dateSwaptionExpiry
        self.cashSettled = True # (voorlopige) aanname in TRT is dat alle swaptions cash settled zijn
        self.getSwaptionType()
        self.getTenor()
        self.getExpiry()
        self.getPriceIMW()

    # Berekenen van class argumenten alleen obv geïnitialiseerde class argumenten
    # Bij het aanmaken van een nieuw object worden deze automatisch aangemaakt.

    def getSwaptionType(self):
        if self.cicCode[2:] == 'B6':
            self.swaptionType = 'payer'
        elif self.cicCode[2:] == 'C6':
            self.swaptionType = 'receiver'
        else:
            raise ValueError('Swaption type could not be found!')

    def getTenor(self):
        self.tenor = (self.dateSwapExpiry - self.dateSwaptionExpiry).days / 365

    def getExpiry(self):
        self.expiry = (self.dateSwaptionExpiry - self.dateIMW).days / 365

    def getPriceIMW(self):
        self.priceIMW = self.marketValueIMW / self.nominalValue

    # Berekenen van class argumenten obv geïnitialiseerde en berekende class argumenten, en obv extra input die niet tot de class behoort
    # Bij het aanmaken van een nieuw object worden deze niet automatisch aangemaakt. 
    # De argumenten worden pas aangemaakt zodra de betreffende functie in de code wordt aangeroepen.
    
    def getMarketValueModel(self, curve):
        self.marketValueModel = getPriceModel(curve, self.swaptionType, self.cashSettled, self.tenor, self.expiry, self.volatilityIMW, self.strike)*self.nominalValue
    
    def getImpliedVolatilityModel(self, curve):
        data = (self.priceIMW, self.swaptionType, self.cashSettled, self.tenor, self.expiry, self.strike, tuple(curve))
        self.impliedVolatilityModel = fsolve(calibrateImpliedVolatilityModel, self.volatilityIMW, data)[0]

    # Class argumenten worden niet aangemaakt, maar berekeningen zijn wel gebaseerd op de bestaande class argumenten, en obv extra input die niet tot de class behoort
    # De berekeningen worden dus niet in de class opgeslagen, dus moeten eventueel elders worden ondergebracht (bv dataframe) voor verder gebruik
    # Dat is in dit geval overigens niet aan de orde, i.e. de expiry wordt slechts eenmalig in de code aangemaakt en is verder niet nodig. 
    # Daarom is het ook niet nodig de berekening als argument op te slaan
    # In principe hoeft onderstaande method niet in de class te staan, maar zou ook als externe functie kunnen worden gedaan.
    # Reden om de method toch in de class te zetten is dat:
            # (ook) gebruik gemaakt wordt van class argumenten
            # de expiry echt een eigenschap is van de class
            # de code kort en overzichtelijk in de class kan worden opgenomen

#     def getYearsToExpirySwaptionFromDatePricing(self, datePricing):
#         yearsToExpirySwaptionFromDatePricing = (self.dateSwaptionExpiry - datePricing).days / 365
#         return yearsToExpirySwaptionFromDatePricing

    # NB. De functie getPriceModel zou in principe ook als onderdeel van de class kunnen worden opgenomen.
    # Net als bij de expiry zou de berekening niet als argument van de class worden opgeslagen.
    # Ditmaal niet omdat de berekening slechts 1x nodig is, maar omdat de berekening voor heel veel curves moet worden gedaan.
    # Het is niet handig al deze berekeningen op te slaan als argumenten van de class, i.e. het is beter om daarvoor een dataframe te gebruiken
    # Reden om de functie niet in de class te zetten is dat:
            # de code relatief omvangrijk is en de class te veel zou vervuilen

### Class voor AoC swaptions

In [20]:
class SwaptionAoC:
    
    def __init__(self, company, portfolio, securityId, swaptionType, cashSettled, strike, tenor, expiryPrimo, marketValueIMWPrimo, nominalValuePrimo, volatilityIMWPrimo, expiryUltimo, marketValueIMWUltimo, nominalValueUltimo, volatilityIMWUltimo):
        self.company = company
        self.portfolio = portfolio
        self.securityId = securityId
        self.swaptionType = swaptionType
        self.cashSettled = cashSettled
        self.strike = strike
        self.tenor = tenor
        self.expiryPrimo = expiryPrimo
        self.marketValueIMWPrimo = marketValueIMWPrimo
        self.nominalValuePrimo = nominalValuePrimo
        self.volatilityIMWPrimo = volatilityIMWPrimo
        self.expiryUltimo = expiryUltimo
        self.marketValueIMWUltimo = marketValueIMWUltimo
        self.nominalValueUltimo = nominalValueUltimo
        self.volatilityIMWUltimo = volatilityIMWUltimo  
        self.getAoCExpiry()
        
    def getAoCExpiry(self):
        pricePrimo = getPriceModel(curveBasePrimo, self.swaptionType, self.cashSettled, self.tenor, self.expiryPrimo, self.volatilityIMWPrimo, self.strike)*self.nominalValuePrimo
        priceUltimo = getPriceModel(curveBasePrimo, self.swaptionType, self.cashSettled, self.tenor, self.expiryUltimo, self.volatilityIMWPrimo, self.strike)*self.nominalValuePrimo        
        self.AoCExpiry = priceUltimo-pricePrimo


## Aanmaken van alle objecten en argumenten voor de class 'Swaption'

### Aanmaken van alle swaption objecten en de initiële argumenten

#### Swaptions primo

In [21]:
swaptionsPrimo = []
for index, row in df_swaptionsIMWPrimo.iterrows():
    swaption = Swaption(row['dateIMW'], row['company'], row['portfolio'], row['cicCode'], row['securityId'], row['marketValueIMW'], row['nominalValue'], row['volatilityPercentageIMW'], row['strikePercentage'], row['dateSwapExpiry'], row['dateSwaptionExpiry'])
    swaptionsPrimo.append(swaption)

#### Swaptions ultimo

In [22]:
swaptionsUltimo = []
for index, row in df_swaptionsIMWUltimo.iterrows():
    swaption = Swaption(
                    row['dateIMW'], \
                    row['company'], \
                    row['portfolio'], \
                    row['cicCode'], \
                    row['securityId'], \
                    row['marketValueIMW'], \
                    row['nominalValue'], \
                    row['volatilityPercentageIMW'], \
                    row['strikePercentage'], \
                    row['dateSwapExpiry'], \
                    row['dateSwaptionExpiry'] \
                    )
    swaptionsUltimo.append(swaption)

### Berekenen van het argument 'market value model' voor alle swaption objecten

#### Market value model primo

In [23]:
curveBasisPrimo = df_curvesPrimo[curveNameBasis].to_list()

In [24]:
for i in range(len(swaptionsPrimo)):
    swaptionsPrimo[i].getMarketValueModel(curveBasisPrimo)

#### Market value model ultimo

In [25]:
curveBasisUltimo = df_curvesUltimo[curveNameBasis].to_list()

In [26]:
for i in range(len(swaptionsUltimo)):
    swaptionsUltimo[i].getMarketValueModel(curveBasisUltimo)

### Berekenen van het argument 'implied volatility' voor alle swaption objecten

#### Implied volatility primo

In [27]:
for i in range(len(swaptionsPrimo)):
    swaptionsPrimo[i].getImpliedVolatilityModel(curveBasisPrimo)

C:\Users\odries\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\optimize\_minpack_py.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


#### Implied volatility ultimo

In [28]:
for i in range(len(swaptionsUltimo)):
    swaptionsUltimo[i].getImpliedVolatilityModel(curveBasisUltimo)

## Aanmaken van alle objecten en argumenten voor de class 'SwaptionAoC'

### Merge de primo en ultimo swaption objecten

In [29]:
columnNames = ['key', 'swaptionsPrimo']
df_swaptionsPrimo = pd.DataFrame(columns = columnNames)
for i in range(len(swaptionsPrimo)):
    key = swaptionsPrimo[i].company+swaptionsPrimo[i].portfolio+swaptionsPrimo[i].securityId
    df_swaptionsPrimo.loc[i, ['key', 'swaptionsPrimo']] = [key, swaptionsPrimo[i]]
#df_swaptionsPrimo

In [30]:
columnNames = ['key', 'swaptionsUltimo']
df_swaptionsUltimo = pd.DataFrame(columns = columnNames)
for i in range(len(swaptionsUltimo)):
    key = swaptionsUltimo[i].company+swaptionsUltimo[i].portfolio+swaptionsUltimo[i].securityId
    df_swaptionsUltimo.loc[i, ['key', 'swaptionsUltimo']] = [key, swaptionsUltimo[i]]
#df_swaptionsUltimo

In [31]:
df_swaptions = pd.merge(df_swaptionsPrimo, df_swaptionsUltimo, on='key', how='outer')
df_swaptions

,key,swaptionsPrimo,swaptionsUltimo
0,2523_ECAPBELAMP3350972,<__main__.Swaption object at 0x0000015A2153A620>,<__main__.Swaption object at 0x0000015A2153AE60>
1,2523_ECAPBELAMP3350574,<__main__.Swaption object at 0x0000015A2153A8F0>,<__main__.Swaption object at 0x0000015A2153A2F0>
2,2523_ECAPBEL1773350415,<__main__.Swaption object at 0x0000015A21539D20>,<__main__.Swaption object at 0x0000015A21539D50>
3,2523_ECAPBELAMP3350971,<__main__.Swaption object at 0x0000015A2153A1D0>,<__main__.Swaption object at 0x0000015A21538700>
4,2523_ECAPBELAMP3350980,<__main__.Swaption object at 0x0000015A21539CF0>,<__main__.Swaption object at 0x0000015A2153BC40>
...,...,...,...
48,2523_ECAPBELAMP3350756,<__main__.Swaption object at 0x0000015A21538DF0>,<__main__.Swaption object at 0x0000015A215395D0>
49,2523_ECAPBELAMP3350982,<__main__.Swaption object at 0x0000015A21538D30>,<__main__.Swaption object at 0x0000015A2153B880>
50,2523_ECAPBELAMP3350983,<__main__.Swaption object at 0x0000015A21539C90>,<__main__.Swaption object at 0x0000015A215389A0>
51,2523_ECAPBELAMP3350967,<__main__.Swaption object at 0x0000015A21538D60>,<__main__.Swaption object at 0x0000015A2153B850>


### Aanmaken van alle swaption objecten en de initiële argumenten

In [32]:
curveBasePrimo = df_curvesPrimo[curveNameBasis].to_list()
curveBaseUltimo = df_curvesUltimo[curveNameBasis].to_list()

In [33]:
swaptions = []
for index, row in df_swaptions.iterrows():
    swaption = SwaptionAoC( \
                           row['swaptionsPrimo'].company, \
                           row['swaptionsPrimo'].portfolio, \
                           row['swaptionsPrimo'].securityId, \
                           row['swaptionsPrimo'].swaptionType, \
                           row['swaptionsPrimo'].cashSettled, \
                           row['swaptionsPrimo'].strike, \
                           row['swaptionsPrimo'].tenor, \
                           row['swaptionsPrimo'].expiry, \
                           row['swaptionsPrimo'].marketValueIMW, \
                           row['swaptionsPrimo'].nominalValue, \
                           row['swaptionsPrimo'].volatilityIMW, \
                           row['swaptionsUltimo'].expiry, \
                           row['swaptionsUltimo'].marketValueIMW, \
                           row['swaptionsUltimo'].nominalValue, \
                           row['swaptionsUltimo'].volatilityIMW \
                          )
    swaptions.append(swaption)    

In [34]:
print(swaptions[0].marketValueIMWPrimo,swaptions[0].marketValueIMWUltimo)

2332319.63 2604833.12


In [35]:
print(swaptions[0].expiryPrimo,swaptions[0].expiryUltimo)

16.18082191780822 16.0986301369863


In [36]:
print(swaptions[0].volatilityIMWPrimo,swaptions[0].volatilityIMWUltimo)

0.00581157662 0.0058942069500000005


In [45]:
print(swaptions[3].AoCExpiry)

-12308.307982577942


## Output

### Berekenen van alle swaption waarden voor alle curves

#### Waardering swaptions primo

In [38]:
columnNamesPrimo = ['company', 'portfolio', 'securityId'] + curveNamesPrimo
df_swaptionMarketValuesModelPrimo = pd.DataFrame(columns = columnNamesPrimo)
for i in range(len(swaptionsPrimo)):
    df_swaptionMarketValuesModelPrimo.loc[i, ['company', 'portfolio', 'securityId']] =  [swaptionsPrimo[i].company, swaptionsPrimo[i].portfolio, swaptionsPrimo[i].securityId]
    for curveNamePrimo in curveNamesPrimo:
        curvePrimo = df_curvesPrimo[curveNamePrimo].to_list()
        df_swaptionMarketValuesModelPrimo.loc[i, curveNamePrimo] = getPriceModel(curvePrimo, swaptionsPrimo[i].swaptionType, swaptionsPrimo[i].cashSettled, swaptionsPrimo[i].tenor, swaptionsPrimo[i].expiry, swaptionsPrimo[i].volatilityIMW, swaptionsPrimo[i].strike) * swaptionsPrimo[i].nominalValue \
                                                            + (swaptionsPrimo[i].marketValueIMW - swaptionsPrimo[i].marketValueModel)

In [39]:
df_swaptionMarketValuesModelPrimo

,company,portfolio,securityId,FairValue,swap.cra.zero.va.down
0,2523_ECAP,BELAMP,3350972,"2,332,319.63","2,182,482.76"
1,2523_ECAP,BELAMP,3350574,"965,615.72","1,120,527.73"
2,2523_ECAP,BEL177,3350415,"1,454,048.72","1,686,098.61"
3,2523_ECAP,BELAMP,3350971,"3,485,393.65","3,286,040.26"
4,2523_ECAP,BELAMP,3350980,"1,790,950.35","1,680,081.62"
...,...,...,...,...,...
48,2523_ECAP,BELAMP,3350756,"86,639.40","44,454.96"
49,2523_ECAP,BELAMP,3350982,"2,406,877.36","2,255,811.66"
50,2523_ECAP,BELAMP,3350983,"2,719,139.91","2,553,753.45"
51,2523_ECAP,BELAMP,3350967,"5,318,994.74","4,969,416.03"


In [40]:
# df_swaptionMarketValuesModelPrimo.to_excel('output.xlsx')

#### Waardering swaptions ultimo

In [41]:
columnNamesUltimo = ['company', 'portfolio', 'securityId'] + curveNamesUltimo
df_swaptionMarketValuesModelUltimo = pd.DataFrame(columns = columnNamesUltimo)
for i in range(len(swaptionsUltimo)):
    df_swaptionMarketValuesModelUltimo.loc[i, ['company', 'portfolio', 'securityId']] =  [swaptionsUltimo[i].company, swaptionsUltimo[i].portfolio, swaptionsUltimo[i].securityId]
    for curveNameUltimo in curveNamesUltimo:
        curveUltimo = df_curvesUltimo[curveNameUltimo].to_list()
        df_swaptionMarketValuesModelUltimo.loc[i, curveNameUltimo] = getPriceModel(curveUltimo, swaptionsUltimo[i].swaptionType, swaptionsUltimo[i].cashSettled, swaptionsUltimo[i].tenor, swaptionsUltimo[i].expiry, swaptionsUltimo[i].volatilityIMW, swaptionsUltimo[i].strike) * swaptionsUltimo[i].nominalValue \
                                                            + (swaptionsUltimo[i].marketValueIMW - swaptionsUltimo[i].marketValueModel)

In [42]:
df_swaptionMarketValuesModelUltimo

,company,portfolio,securityId,FairValue,swap.cra.zero.va.down
0,2523_ECAP,BELAMP,SWOP3350992,"4,691,587.59","4,577,293.98"
1,2523_ECAP,BELAMP,3350967,"5,962,038.17","5,425,207.97"
2,2523_ECAP,BELAMP,3350952,"7,356,626.75","6,651,239.66"
3,2523_ECAP,BELAMP,3350974,"2,300,855.42","2,112,350.04"
4,2523_ECAP,BELAMP,3350955,"6,473,481.65","5,863,307.91"
...,...,...,...,...,...
48,2523_ECAP,BELAMP,SWOP3351024,"6,965,135.77","6,784,030.43"
49,2523_ECAP,BELAMP,3350962,"10,223,462.83","9,312,122.44"
50,2523_ECAP,BELAMP,3350970,"2,851,131.09","2,607,380.02"
51,2523_ECAP,BELAMP,SWOP3351010,"9,370,676.53","9,145,566.92"


### AoC van fair value swaptions

## Verificatie